In [1]:
import hsman
import os
import xarray
import rioxarray
import xmltodict
import geopandas
from shapely.geometry import Polygon

In [2]:
# clip large dataset to area of annotation
gdf1 = geopandas.read_file('data/Milton_Keynes_labels.gpkg')

# internal data manager
hsman.get_datasets().set_index('ID').loc['OUMI1202006221']

,index,dataset,geometry,Site,type,date
ID,,,,,,
OUMI1202006221,54,OUMI1202006221_DSM_aerial,"POLYGON ((-0.83758 52.04093, -0.69362 52.03820...",OUMI1,DSM,2020-06-22
OUMI1202006221,33,OUMI1202006221_DTM_aerial,"POLYGON ((-0.83758 52.04093, -0.69362 52.03820...",OUMI1,DTM,2020-06-22
OUMI1202006221,28,OUMI1202006221_RGB_aerial,"POLYGON ((-0.83758 52.04092, -0.69348 52.03819...",OUMI1,RGB,2020-06-22
OUMI1202006221,64,OUMI1202006221_SWIR_aerial,"POLYGON ((-0.83587 52.03894, -0.69733 52.03632...",OUMI1,SWIR,2020-06-22
OUMI1202006221,21,OUMI1202006221_VNIR_aerial,"POLYGON ((-0.83612 52.03896, -0.69738 52.03632...",OUMI1,VNIR,2020-06-22


In [3]:
s2_root = '/STEM/data/project/Treeview/satellite/sentinel2/'
os.listdir(s2_root)

['S2B_MSIL1C_20211202T111329_N0301_R137_T30UXC_20211202T120633.SAFE',
 'S2B_MSIL2A_20211202T111329_N0301_R137_T30UXC_20211202T125422.SAFE',
 'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE',
 'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE',
 '.ipynb_checkpoints']

In [4]:
def get_meta_data(fpath):
    # try 2A
    try:
        with open(fpath, 'r') as f:
            return xmltodict.parse(f.read())
    except FileNotFoundError:
        try:
            _fpath = os.path.join(fpath, 'MTD_MSIL2A.xml')
            with open(_fpath, 'r') as f:
                return xmltodict.parse(f.read())
        except FileNotFoundError:
            _fpath = os.path.join(fpath, 'MTD_MSIL1C.xml')
            with open(_fpath, 'r') as f:
                return xmltodict.parse(f.read())
            
def get_angles(fpath):
    def _parse(entry):
        return float(entry['#text'])
    angle_dict = get_meta_data(fpath)['n1:Level-1C_Tile_ID']['n1:Geometric_Info']['Tile_Angles']
    angles = {}
    for k in ['ZENITH_ANGLE', 'AZIMUTH_ANGLE']:
        angles['SOLAR_'+k] = _parse(angle_dict['Mean_Sun_Angle'][k])
        angles['VIEW_'+k] = _parse(angle_dict['Mean_Viewing_Incidence_Angle_List']['Mean_Viewing_Incidence_Angle'][0][k])
    return angles

In [5]:
A = get_angles(os.path.join(s2_root,
                            'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE',
                            'GRANULE','L1C_T30UXC_A026160_20200625T111446','MTD_TL.xml'))
A

{'SOLAR_ZENITH_ANGLE': 29.9793624960938,
 'VIEW_ZENITH_ANGLE': 3.06889419958891,
 'SOLAR_AZIMUTH_ANGLE': 157.124519128112,
 'VIEW_AZIMUTH_ANGLE': 173.991655773712}

In [6]:
# buffer it by 50m
bounds_S2A = Polygon.from_bounds(*gdf1.total_bounds).buffer(50).bounds

In [7]:
s2a_1c_paths = [
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B02.jp2',
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B03.jp2',
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B04.jp2',
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B08.jp2'
]

s2a_2a_paths = [
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R10m/T30UXC_20200625T110631_B02_10m.jp2',
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R10m/T30UXC_20200625T110631_B03_10m.jp2',
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R10m/T30UXC_20200625T110631_B04_10m.jp2',
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R10m/T30UXC_20200625T110631_B08_10m.jp2'
]

s2a_1c_paths_20 = [
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B05.jp2',
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B06.jp2',
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B07.jp2',
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B8A.jp2',
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B11.jp2',
    'S2A_MSIL1C_20200625T110631_N0209_R137_T30UXC_20200625T114101.SAFE/GRANULE/L1C_T30UXC_A026160_20200625T111446/IMG_DATA/T30UXC_20200625T110631_B12.jp2',
]

s2a_2a_paths_20 = [
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R20m/T30UXC_20200625T110631_B05_20m.jp2',
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R20m/T30UXC_20200625T110631_B06_20m.jp2',
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R20m/T30UXC_20200625T110631_B07_20m.jp2',
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R20m/T30UXC_20200625T110631_B8A_20m.jp2',
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R20m/T30UXC_20200625T110631_B11_20m.jp2',
    'S2A_MSIL2A_20200625T110631_N0214_R137_T30UXC_20200625T122333.SAFE/GRANULE/L2A_T30UXC_A026160_20200625T111446/IMG_DATA/R20m/T30UXC_20200625T110631_B12_20m.jp2',
]

def load_data(paths, bands, root=s2_root, bounds=bounds_S2A):
    arrs = []
    for p in paths:
        arrs.append(
            rioxarray.open_rasterio(os.path.join(root, p)).rio.clip_box(*bounds_S2A,)
        )
    return xarray.concat(arrs, dim='band').assign_coords(band=bands)

s2a_l1c_10m = load_data(s2a_1c_paths, [0,1,2,6])
s2a_l2a_10m = load_data(s2a_2a_paths, [0,1,2,6])

s2a_l1c_20m = load_data(s2a_1c_paths_20, [3, 4, 5, 7, 8, 9])
s2a_l2a_20m = load_data(s2a_2a_paths_20, [3, 4, 5, 7, 8, 9])

In [8]:
band_names = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11','B12']
s2a_l1c = xarray.Dataset(
    {'reflectance': xarray.concat([ s2a_l1c_10m, s2a_l1c_20m.interp_like(s2a_l1c_10m.isel(band=0))],
                                  dim='band').sortby('band').astype('uint16')},
    coords = {'band_name': ('band', band_names)}
)

s2a_l2a = xarray.Dataset(
    {'reflectance': xarray.concat([ s2a_l2a_10m, s2a_l2a_20m.interp_like(s2a_l2a_10m.isel(band=0))],
                                  dim='band').sortby('band').astype('uint16')},
    coords = {'band_name': ('band', band_names)}
)

In [9]:
s2a_l1c.to_netcdf(
    'data/Milton_Keynes_Sentinel2_L1C.nc'
)

s2a_l2a.to_netcdf(
    'data/Milton_Keynes_Sentinel2_L2A.nc'
)

In [10]:
# df1 = hsman.open_dataset('OUMI1202006221_VNIR_aerial')
# df2 = hsman.open_dataset('OUMI1202006221_SWIR_aerial')


# df1_mk_1 = df1.rio.clip_box(*gdf1.total_bounds).astype('uint16').compute()
# df2_mk_1 = df2.rio.clip_box(*gdf1.total_bounds).astype('uint16').compute()

# df1_mk_1.to_netcdf('data/mk_1_vnir.nc')
# df2_mk_1.to_netcdf('data/mk_1_swir.nc')